# Multiprocessing and Multithreading

#### cloned from https://github.com/brianjp93/Multiprocessing-tuts

###### Goal
<ul>
<li>Speed up code by using multiple processes</li>
</ul>

###### Options
<ul>
<li>Multithreading</li>
<li>Multiprocessing</li>
</ul>

#### Multithreading

Can use when
<ul>
<li>Lots time waiting around for a response</li>
<ul>
<li>Network Requests - http get, post, put</li>
</ul>
<li>
Lots of I/O (Read, Write, Send, Recv...)
</li>
</ul>

##### Still Bound by Global Interpreter Lock

## CPU Bound Threading

In [1]:
from __future__ import division
from threading import Thread
import multiprocessing
from multiprocessing import Process
import time

Make a list with 10 million 10's.

In [2]:
myLen = 10000000*5
myList = [10]*myLen
myList[:15]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [3]:
def squared(num):
    num**2

def squareList(lst):
    for i in lst:
        squared(i)    

# Square all of the numbers in the list.

Just a regular for loop.

In [ ]:
def p_plain(list_len):
    myLen = list_len
    myList = [10]*myLen
    
    start = time.time() #  Get current time
    squareList(myList)
    serialprocesstime = time.time() - start
    print("Squaring 10 million numbers took {} seconds.".format(round(serialprocesstime,2)))
    
    return serialprocesstime

# Squaring with multiprocessing

Let's do the same squaring function we did before, but this time with multiprocessing

According to the python docs, pool cannot be used in the interactive interpreter.

It seems that this extends to an ipython notebook.
<a href="https://docs.python.org/3/library/multiprocessing.html#using-a-pool-of-workers">python pool docs</a>

In [ ]:
def p_multi(list_len):
    num_processes = multiprocessing.cpu_count() # number of cores?
    
    process_list = [] # jobs to be run simultaneously
    
    # recreate a list of 10 mil numbers
    myList = [10]*(list_len//num_processes) # divide the iterable into the number of cores available
    
    for p in range(num_processes):
        p = Process(target=squareList, args=(myList,)) # create two iterations of your target function ("target=myFunc")
        process_list.append(p)

    start = time.time()
    for p in process_list:
        p.start()

    for p in process_list:
        p.join() # unsure, but maybe puts everything back together again

    squareprocesstime = time.time() - start
    print("Squaring 10 million numbers took {} seconds with {} processes.".format(round(squareprocesstime,2),num_processes))
    
    return squareprocesstime

# Limitations

Length of job needs to be long enough so that the cost of setting up all the multiprocessing
can be outweighed by the speed increases.

e.g., 
    - squaring a list of 10K numbers will be slightly slower with mp
    - squaring a list of 100K numbers will be about even
    - squaring a list of 1M numbers or more and the mp will begin to be faster
    - perhaps the max speed boost is the number of cores? (e.g. 2x boost)

In [ ]:
myLen = 15**6
speed_increase = p_plain(myLen) / p_multi(myLen)
print(f"Multiprocessing was {round(speed_increase,2)}X faster.")

# Test with file read/write

In [4]:
lenin = "Lenin's Bolshevik government initially shared power with the Left Socialist Revolutionaries, elected soviets, and a multi-party Constituent Assembly, although by 1918 it had centralised power in the new Communist Party. Lenin's administration redistributed land among the peasantry and nationalised banks and large-scale industry. It withdrew from the First World War by signing a treaty with the Central Powers and promoted world revolution through the Communist International. Opponents were suppressed in the Red Terror, a violent campaign administered by the state security services; tens of thousands were killed or interned in concentration camps. His administration defeated right and left-wing anti-Bolshevik armies in the Russian Civil War from 1917 to 1922 and oversaw the Polish–Soviet War of 1919–1921. Responding to wartime devastation, famine, and popular uprisings, in 1921 Lenin encouraged economic growth through the market-oriented New Economic Policy. Several non-Russian nations secured independence after 1917, but three re-united with Russia through the formation of the Soviet Union in 1922. In increasingly poor health, Lenin expressed opposition to the growing power of his successor, Joseph Stalin, before dying at his dacha in Gorki."
lenin = lenin.split(" ")

In [20]:
def writer(lst,filename):
    with open(filename,"a") as file:
        for n in range(1000):
            for word in lst:
                file.write(word + " ")

In [6]:
def write_plain(num_loops):
    
    start = time.time()
    
    for n in range(num_loops):
        writer(lenin)

    processtime = time.time() - start
    
    return processtime

In [7]:
def write_multi(num_loops):
    
    process_list_1 = []
    process_list_2 = []
    # need to append HALF the writer()'s to list_1, the other half to list_2 (one process per core)
    
    for p in range(num_loops):
        p = Process(target=writer, args=(lenin,))
        process_list.append(p)

    start = time.time()
    for p in process_list:
        p.start()

    for p in process_list:
        p.join()
    processtime = time.time() - start
    return processtime

In [38]:
def write_plain_2(num_loops):
    start = time.time()
    for n in range(num_loops):
        writer(lenin,"write_plain.txt")
    processtime = time.time() - start
    return processtime

In [40]:
def write_multi_2(num_loops):
    start = time.time()
    p_list = []
    for n in range(num_loops):
        p = Process(target=writer, args=(lenin,"write_multi.txt",))
        p_list.append(p)
    for p in p_list:
        p.start()
    processtime = time.time() - start
    return processtime

In [60]:
n_times = 2
speed_increase = write_plain_2(n_times) / write_multi_2(n_times)
print(f"Multiprocessing was {round(speed_increase,2)}X faster.")

Multiprocessing was 18.91X faster.


# Test with PGN parsing